## 6.5 Dynamic arrays

A **dynamic array** is a data type that adds a resize operation
to a static array data structure.
The operation creates a new static array of the length asked for,
copies the items from the current to the new static array,
which then becomes the current one. The old static array is garbage collected.

If the length is increased, the extra positions are filled with the default
value for the type of items in the static array.
If the length is decreased, the excess items are lost.
For example, if a dynamic array with integers 3, 5, 7, 9, 11 is resized to
a length of three, the new static array has values 3, 5, 7.
It's up to dynamic array users to resize them when appropriate
for the problem at hand.

A dynamic array has the main advantage of static arrays,
i.e. constant-time access and replacement of items,
with the flexibility of growing and shrinking as necessary.
This is achieved through a succession of static arrays of different lengths.

<div class="alert alert-info">
<strong>Info:</strong> In some programming languages, a dynamic array is known as a vector.
</div>

Shrinking an array can be done in constant time,
by simply setting the length to a smaller value to truncate the array,
without any copying. This requires memory management support in order to
garbage collect part of an existing array,
which may not be available on all platforms.

It's thus best to assume the complexity of the resize operation is always
linear in the new length, because the operation has to create and initialise
a new array of that length and then copy at most that many items to it.

### 6.5.1 The `DynamicArray` class

Since a dynamic array can be seen as a kind of static array,
I can implement dynamic arrays as a subclass of `StaticArray`,
which is defined in `m269_array.py`.

In [1]:
%run -i ../m269_array

Subclasses inherit all the operations of their parent class,
so I only have to add the resize operation.
Methods of subclasses can (and usually have to) access instance variables
of their superclass.

The postconditions for the resize operation must state that
the new array keeps the same items up to the shorter of the two arrays and
the rest, if any, are `None`.
I use again the 'pre-*x*' and 'post-*x*' notation to refer to the
value of variable *x* before and after the operation.

In [2]:
# this code is also in m269_array.py


class DynamicArray(StaticArray):
    """An array that can grow and shrink."""

    def resize(self, length: int) -> None:
        """Shorten or extend the array to the new length.

        Preconditions: 0 <= length; length != self.length()
        Postconditions: if pre-self is a_0, a_1, ..., a_n then
        post-self is b_0, b_1, ..., b_m with
        - n == pre-self.length() - 1
        - m == length - 1
        - b_i == a_i for every i from 0 to min(n, m)
        - b_i == None for every i from min(m, n) + 1 to m
        """
        new_array = [None] * length
        for index in range(0, min(length, self.length())):
            new_array[index] = self.items[index]
        self.items = new_array

Note that I can call the `length` method on the dynamic array `self`
even though the method isn't defined in this class.
Every instance of `DynamicArray` is an instance of `StaticArray`,
so all methods of the latter also apply to the former.

### 6.5.2 Tests

The black-box testing of the new method is done as usual:
create an array with 0, 1, 2, ..., resize it and check the result with the
`length`  and `get_item` methods.
After resizing, each value is either the same or `None`,
as indicated in the postconditions above.

In [3]:
%run -i ../m269_test


def test_resize(old_length: int, new_length: int) -> None:
    """Test resizing a dynamic array from old_length to new_length.

    Preconditions:
    - 0 <= old_length; 0 <= new_length
    - old_length != new_length
    """
    array = DynamicArray(old_length)
    for index in range(old_length):
        array.set_item(index, index)
    array.resize(new_length)
    check("new length", array.length(), new_length, array)
    for index in range(new_length):
        if index < min(old_length, new_length):
            value = index  # old item
        else:
            value = None  # new item
        check("get item", array.get_item(index), value, array)

I must test both growing and shrinking.
The edge cases are resizing from and to an array of length zero or one,
the smallest non-empty array.

In [4]:
for old in (0, 1, 3, 6):
    for new in (0, 1, 3, 6):
        if old != new:
            print("Resize from", old, "to", new)
            test_resize(old, new)

Resize from 0 to 1
Resize from 0 to 3
Resize from 0 to 6
Resize from 1 to 0
Resize from 1 to 3
Resize from 1 to 6
Resize from 3 to 0
Resize from 3 to 1
Resize from 3 to 6
Resize from 6 to 0
Resize from 6 to 1
Resize from 6 to 3


⟵ [Previous section](06_4_bounded.ipynb) | [Up](06-introduction.ipynb) | [Next section](06_6_use_dyn_array.ipynb) ⟶